## Login Block

In [ ]:
from kiteconnect import KiteConnect
import ipywidgets as widgets
from IPython.display import display

def kite_login(api_key: str, api_secret: str):
    """
    Opens Kite login, accepts request_token, generates access_token,
    sets it on KiteConnect object and returns the kite instance.
    """

    kite = KiteConnect(api_key=api_key)
    auth_url = kite.login_url()

    # ---- Widgets ----
    login_link = widgets.HTML(
        value=f"""
        <a href="{auth_url}" target="_blank"
           style="background:#2563eb;color:white;
                  padding:4px 12px;border-radius:4px;
                  text-decoration:none;font-size:13px;">
            🔓 Kite Login
        </a>
        """
    )

    token_box = widgets.Text(
        placeholder="Paste request_token here",
        layout=widgets.Layout(width="320px")
    )

    save_btn = widgets.Button(
        description="Login",
        layout=widgets.Layout(width="80px"),
        button_style="success"
    )

    status = widgets.HTML("")

    # ---- Callback ----
    def do_login(b):
        try:
            request_token = token_box.value.strip()
            data = kite.generate_session(
                request_token=request_token,
                api_secret=api_secret
            )
            access_token = data["access_token"]
            kite.set_access_token(access_token)

            status.value = "<small>✅ Kite login successful</small>"
        except Exception as e:
            status.value = f"<small style='color:red;'>❌ {e}</small>"

    save_btn.on_click(do_login)

    # ---- Layout ----
    display(widgets.HBox([login_link, token_box, save_btn]))
    display(status)

    return kite


## Chart Monitor

In [ ]:
from datetime import date, timedelta
import threading
import time
import ipywidgets as widgets
from IPython.display import display

# ------------------- UDF -------------------
def check_last_candle_strength(
    kite,
    instrument_token=256265,
    interval="15minute",
    lookback_days=2,
    threshold_percent=75
):
    to_date = date.today()
    from_date = to_date - timedelta(days=lookback_days)

    data = kite.historical_data(
        instrument_token=instrument_token,
        interval=interval,
        from_date=from_date,
        to_date=to_date,
        continuous=False
    )

    if not data:
        print("No data received")
        return

    last_data = data[-1] ## -1 Gets current candle even before closing and -2 gets last closed 15 minutes candle's 

    body_size = abs(last_data['close'] - last_data['open'])
    candle_range = last_data['high'] - last_data['low']

    if candle_range == 0:
        print("Invalid candle range")
        return

    body_percent = (body_size / candle_range) * 100
    open_price = last_data['open']
    close_price = last_data['close']

    # print("Last candle:", last_data)
    print("Body %:", round(body_percent, 2))

    if body_percent >= threshold_percent:
        print("🔻 Strong Trend Detected!")
        alert_mail(subject = "Strong Trend Alert Mail", body = "Strong Trend Alert Mail")
        
    elif body_percent <= 25 and close_price <= open_price * 0.75:
        print("🔻 Strong Bearish Candle Detected!")
        alert_mail(subject = "Strong Bearish Trend", body = "Strong Bearish Trend")
        
    elif body_percent <= 25 and close_price >= open_price * 1.25:
        print("🔺 Strong Bullish Candle Detected!")
        alert_mail(subject = "Strong Bullish Trend", body = "Strong Bullish Trend")
    else:
        print("❌ Not Strong candle")


# ------------------- SCHEDULER -------------------
stop_event = threading.Event()
checker_thread = None

def candle_checker_loop():
    while not stop_event.is_set():
        try:
            check_last_candle_strength(kite)
        except Exception as e:
            print("Error:", e)

        # responsive sleep (5 minutes total)
        for _ in range(300):
            if stop_event.is_set():
                break
            time.sleep(1)


# ------------------- START -------------------
def start_monitor():
    global checker_thread
    if checker_thread is None or not checker_thread.is_alive():
        stop_event.clear()
        checker_thread = threading.Thread(
            target=candle_checker_loop,
            daemon=True
        )
        checker_thread.start()
        print("✅ Candle checker started (every 3 minutes)")
    else:
        print("⚠️ Candle checker already running")


# ------------------- STOP -------------------
def stop_monitor():
    global checker_thread
    if checker_thread and checker_thread.is_alive():
        stop_event.set()
        checker_thread.join(timeout=5)
        print("🛑 Candle checker stopped")
    else:
        print("⚠️ No active candle checker")


# ------------------- UI: SINGLE TOGGLE SWITCH -------------------
toggle = widgets.ToggleButton(
    value=False,
    description="Monitor OFF",
    button_style="danger",
    icon="stop",
    tooltip="Toggle candle monitor"
)

def toggle_handler(change):
    if change["new"]:  # ON
        toggle.description = "Monitor ON"
        toggle.button_style = "success"
        toggle.icon = "check"
        start_monitor()
    else:              # OFF
        toggle.description = "Monitor OFF"
        toggle.button_style = "danger"
        toggle.icon = "stop"
        stop_monitor()

toggle.observe(toggle_handler, names="value")


## Auto Mailer

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def alert_mail(subject = "Strong Trend Alert Mail", body = "Strong Trend Alert Mail"):
    # Email configuration
    sender_email = "prasannanjaneyulu.bandaru@gmail.com"
    receiver_email = "prasannanjaneyulu.bandaru@gmail.com"  # sending to yourself
    password = "bqmt gtld cecs zmcq"# consider using an App Password for Gmail


    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # Connect to the Gmail SMTP server and send the email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        server.quit()
